In [ ]:
pip install pandas

In [ ]:
import pandas as pd
import csv

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#Criando um dicionário com todos os caminhos dos arquivos. Isso vai ser bom porque utilizarei as chaves para criar uma coluna de "Ano"
arquivos = {
    '2019': 'notebooks/data_old/func2019_completo.csv',
    '2020': 'notebooks/data_old/func2020_completo.csv',
    '2021': 'notebooks/data_old/func2021_completo.csv'
}

lista_dataframes = [] #Para armazenar todos os dataframes

for ano, arquivo in arquivos.items():
    try:

        # Como mostrado na monitoria, conseguimos identificar o problema interno do .csv
        # Cada linha do arquivo estava dentro de "...", então o panda estava lendo tudo como uma variavel do tipo string
        # e Desconsiderando as virgulas internas.
        # Conseguimos resolver utilizando o atributo quoting, que ignora o padrão do pandas de ignorar as virgulas dentro das strings
        df = pd.read_csv(arquivo, sep=',', quoting=csv.QUOTE_NONE, engine='python', on_bad_lines='skip')

        # O uso de on_bad_lines='skip' se deu pelo fato dos arquivos não estarem padronizados e devido o tratamente anterior
        # Conferimos que as tuplas ignoradas não alteram nosso ETL nem EDA.
        # O problema: A primeira coluna vai ter uma aspa no início
        # e a última coluna vai ter uma aspa no final. Vamos limpar:

        # Pega o nome da primeira e última coluna
        col_primeira = df.columns[0]
        col_ultima = df.columns[-1]

        # Limpa os dados dessas colunas
        df[col_primeira] = df[col_primeira].str.replace('"', '')
        df[col_ultima] = df[col_ultima].str.replace('"', '')

        # Limpa também os nomes das colunas no cabeçalho
        df.columns = df.columns.str.replace('"', '')

        # Salvando esse pré-processamente, pois é um problema que só conseguimos contornar de forma interna ao pandas
        df.to_csv(f'/data/dados_servidores_{ano}.csv', index=False)

        lista_dataframes.append(df)

    except Exception as e:
        print(f"Erro ao ler os dados do ano {ano}: {e}")